In [1]:
import pandas as pd
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
os.chdir('data')
os.makedirs('orca_spectrograms')
os.makedirs('noise_spectrograms')

In [3]:
orca_df = pd.read_csv("orca_data.csv")
noise_df = pd.read_csv("noise_data.csv")
noise_df.shape

(322, 3)

In [4]:
# creating list with orca audio chunks
orca_audio_chunks = []
for index, row in orca_df.iterrows():
    filename = row['wav_filename']
    filepath = 'wav/' + filename
    start_time = row['start_time_s']  # Keep in seconds
    end_time = row['end_time_s']  # Keep in seconds
    
    # Load the original wav file
    y, sr = librosa.load(filepath)

    # Convert start and end times to sample indices
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)

    # Extract the chunk
    chunk = y[start_sample:end_sample]
    orca_audio_chunks.append(chunk)
    
print(len(orca_audio_chunks))

1953


In [5]:
# functions to help convert list of audio chunks to images and export
def generate_mel_spectrogram(audio_chunk, sr=22050, n_mels=128, fmax=8000):
    # Generate the mel spectrogram
    S = librosa.feature.melspectrogram(y=audio_chunk, sr=sr, n_mels=n_mels, fmax=fmax)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # Plot the mel spectrogram
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel', fmax=fmax)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-frequency spectrogram')
    plt.tight_layout()
    
    # Save the plot to a buffer
    plt.savefig('mel_spectrogram.png')
    plt.close()
    
    # Open the saved image and return it
    image = Image.open('mel_spectrogram.png')
    return image

def save_spectrogram_image(image, file_path):
    # Save the image to the specified directory
    image.save(file_path)

In [6]:
# creating spectrograms for orca calls
for i in range(len(orca_audio_chunks)):
    mel_spectrogram_image = generate_mel_spectrogram(orca_audio_chunks[i], sr)
    path = 'orca_spectrograms/' + str(i) + '.png'
    save_spectrogram_image(mel_spectrogram_image, path)

In [7]:
# creating list with noise audio chunks
noise_audio_chunks = []
for index, row in noise_df.iterrows():
    filename = row['wav_filename']
    filepath = 'wav/' + filename
    start_time = row['start_time_s']  # Keep in seconds
    end_time = row['end_time_s']  # Keep in seconds
    
    # Load the original wav file
    y, sr = librosa.load(filepath)

    # Convert start and end times to sample indices
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)

    # Extract the chunk
    chunk = y[start_sample:end_sample]
    noise_audio_chunks.append(chunk)
    
print(len(noise_audio_chunks))

322


In [8]:
# creating spectrograms for noise
for i in range(len(noise_audio_chunks)):
    mel_spectrogram_image = generate_mel_spectrogram(noise_audio_chunks[i], sr)
    path = 'noise_spectrograms/' + str(i) + '.png'
    save_spectrogram_image(mel_spectrogram_image, path)